In [12]:
import numpy as np
import pandas as pd
import requests
import json
import ast

import sys
sys.path.append('../src/')
import wiki_api
import data_layer
import graph_api


In [2]:
host = "localhost"
port = "8100"

# T0PP

## Example outputs

In [52]:
get_answer(f"Simple definition of a hyperplane:")

'<pad> A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.</s>'

In [53]:
get_answer(f"What mathematical concepts are mentioned here? A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.")

'<pad> plane, line, space</s>'

In [57]:
get_answer(f"In mathematics, what is matrix?")

'<pad> a two-dimensional array of elements</s>'

In [74]:
get_answer(f"Give me a detailled definition of Kernel (algebra)")

'A kernel is a function that maps a real number to another real number.'

In [62]:
get_answer(f"In linear algebra, what is matrix transpose?")

'<pad> matrix transpose is a matrix operation that transposes a matrix into its identity matrix.</s>'

In [63]:
get_answer(f"Germany // Berlin\nFrance // Paris\nRussia // Moscow\nItaly - Rome\n USA //")

'<pad> Berlin</s>'

## Dep Extraction

In [4]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'].replace("<pad> ","").replace("</s>", "")

In [ ]:
def get_deps(i, concept):
    if i % 5 == 0: print(i)
    definition = get_answer(f"What is the mathematical definition of {concept}?")
    response = get_answer(f"What mathematical concepts are mentioned here: {definition}")
    deps = list(map(lambda x: x.strip(), response.split(",")))
    return deps

_, wiki_concepts = data_layer.read_index_and_wiki_concepts()
wiki_concepts = list(wiki_concepts)

#concepts = np.random.choice(wiki_concepts, 100)
deps = [get_deps(i, concept) for i, concept in enumerate(wiki_concepts)]
df_concepts = pd.DataFrame({"concept": wiki_concepts, "deps": deps})
df_concepts.to_csv("../dat/llm/t0pp_output.csv")

In [13]:
#df_concepts = pd.read_csv("../dat/llm/t0pp_output.csv")
#df_concepts.deps = df_concepts.deps.apply(ast.literal_eval)

In [ ]:
link_set = set(wiki_concepts)
df_concepts['dep_articles'] = [print(i) or [wiki_api.disambiguate(dep, link_set, "Mathematics") for dep in concept_deps] for i, concept_deps in enumerate(df_concepts.deps)]

In [21]:
df_concepts = pd.read_csv("../dat/llm/df_concepts.csv")
df_concepts.drop(columns=df_concepts.columns[0], axis=1, inplace=True)
df_concepts.deps = df_concepts.deps.apply(ast.literal_eval)
df_concepts.dep_articles = df_concepts.dep_articles.apply(ast.literal_eval)

In [22]:
def delete_reflexive(concept, dep_articles):
    return list(filter(lambda x: x!=concept, dep_articles))
df_concepts.dep_articles = [delete_reflexive(concept, dep_articles) for concept, dep_articles in zip(df_concepts.concept, df_concepts.dep_articles)]

In [25]:
df_concepts.to_csv("../dat/llm/df_concepts.csv")

In [26]:
graph_api.save_graph('../dat/llm/graph.json', df_concepts.concept, df_concepts.dep_articles)

In [ ]:
#print(list(zip(df_concepts.concept, df_concepts.deps, df_concepts.dep_articles)))

# Galactica

In [4]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'] #.replace("<pad> ","").replace("</s>", "")

In [7]:
get_answer(f"This is a list of mathematical concepts that are required to define matrix:\n")

['This is a list of mathematical concepts that are required to define matrix:\n\n* Set\n* Function\n* Cartesian product\n* Cartesian product of sets\n* Cartesian product of functions\n* Cartesian']

In [8]:
get_answer(f"Brief definition of matrix:\n")

['Brief definition of matrix:\n\nA matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\n# 2.2.1.1 Matrix Addition']

In [10]:
get_answer(f"A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:")

['A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:Numbers']

In [19]:
get_answer(f"Brief definition of hyperplane:\n")

['Brief definition of hyperplane:\n\nA hyperplane is a set of points in a vector space that are equidistant from a given point.\n\n\nAnswer:\n\nA hyperplane is a subspace of']

In [22]:
get_answer(f"A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'")

["A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'Vector space"]

In [25]:
get_answer(f"This is a list of concepts in the definition of Convex set:\n\n")

['This is a list of concepts in the definition of Convex set:\n\n* \\(\\forall x,y\\in S:\\forall \\lambda \\in [0,']

In [66]:
indices, wiki_concepts = data_layer.read_index_and_wiki_concepts()

In [99]:
for concept in np.random.choice(wiki_concepts, 20):
    print("CONCEPT: " + concept)
    context = wiki_api.get_first_paragraph(concept)
    response = get_answer(f"{context}\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA:")
    print(response)

CONCEPT: Boundary value problem
['In mathematics, in the field of differential equations, a boundary value problem is a differential equation together with a set of additional constraints, called the boundary conditions.[1] A solution to a boundary value problem is a solution to the differential equation which also satisfies the boundary conditions.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: differential equations, boundary value problem']
CONCEPT: Minimal polynomial (linear algebra)
['In linear algebra, the minimal polynomial μA of an n × n matrix A over a field F is the monic polynomial P over F of least degree such that P(A) = 0.  Any other polynomial Q with Q(A) = 0 is a (polynomial) multiple of μA.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: Polynomials, matrices, fields']
CONCEPT: Linear function (calculus)
['In calculus and related areas of mathematics, a linear function from the real numbers to the real numbers is